In [3]:
# Setup commands to create folder and move schemas
!mkdir -p /content/binder_schemas
!mv /content/schemas/*.json /content/binder_schemas/ 2>/dev/null || true

# Import required libraries
import os
import json
import sqlite3
import re
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# Configure Gemini API key (replace with your key or use environment variable)
genai.configure(api_key=os.getenv("GEMINI_API_KEY", "your_valid_api_key_here"))

# Initialize Gemini model
try:
    model = genai.GenerativeModel("models/gemini-1.5-flash")
    print("Found valid model: models/gemini-1.5-flash")
    print("Using model: models/gemini-1.5-flash")
except Exception as e:
    print(f"Error initializing model: {e}")
    exit(1)

# In-context examples for query generation
in_context_examples = [
    {
        "input": "Which passengers received refunds greater than $400 for cancelled bookings on flights longer than 200 minutes?",
        "binder_program": """
import sqlite3
conn = sqlite3.connect('{db_name}')
cursor = conn.cursor()
cursor.execute('''
    SELECT DISTINCT p.name
    FROM Passengers p
    JOIN Bookings b ON p.passenger_id = b.passenger_id
    JOIN Cancellations c ON b.booking_id = c.booking_id
    JOIN Flights f ON b.flight_id = f.flight_id
    WHERE c.refund_amount_usd > 400
    AND f.duration_minutes > 200
''')
result = [row[0] for row in cursor.fetchall()]
conn.close()
"""
    },
    {
        "input": "Which airlines operated flights booked by passengers who also booked first-class tickets on flights arriving in Singapore?",
        "binder_program": """
import sqlite3
conn = sqlite3.connect('{db_name}')
cursor = conn.cursor()
cursor.execute('''
    SELECT DISTINCT a.name
    FROM Airlines a
    JOIN Flight_Operators fo ON a.airline_id = fo.airline_id
    JOIN Flights f ON fo.flight_id = f.flight_id
    JOIN Bookings b ON f.flight_id = b.flight_id
    JOIN Passengers p ON b.passenger_id = p.passenger_id
    WHERE p.passenger_id IN (
        SELECT b2.passenger_id
        FROM Bookings b2
        JOIN Tickets t ON b2.booking_id = t.booking_id
        JOIN Flights f2 ON b2.flight_id = f2.flight_id
        JOIN Airports ap ON f2.arrival_airport_id = ap.airport_id
        WHERE t.class = 'First' AND ap.city = 'Singapore'
    )
''')
result = [row[0] for row in cursor.fetchall()]
conn.close()
"""
    },
    {
        "input": "Which attendees from the USA purchased tickets for events sponsored by beverage companies and held in venues with capacity over 15000?",
        "binder_program": """
import sqlite3
conn = sqlite3.connect('{db_name}')
cursor = conn.cursor()
cursor.execute('''
    SELECT DISTINCT a.name
    FROM Attendees a
    JOIN Tickets t ON a.attendee_id = t.attendee_id
    JOIN Events e ON t.event_id = e.event_id
    JOIN Venues v ON e.venue_id = v.venue_id
    JOIN Event_Sponsors es ON e.event_id = es.event_sponsor_id
    JOIN Sponsors s ON es.sponsor_id = s.sponsor_id
    WHERE a.country = 'USA'
    AND v.capacity > 15000
    AND s.industry = 'Beverage'
''')
result = [row[0] for row in cursor.fetchall()]
conn.close()
"""
    },
    {
        "input": "Which performers appeared at events attended by the same attendees who purchased merchandise for other events?",
        "binder_program": """
import sqlite3
conn = sqlite3.connect('{db_name}')
cursor = conn.cursor()
cursor.execute('''
    SELECT DISTINCT p.name
    FROM Performers p
    JOIN Event_Performers ep ON p.performer_id = ep.performer_id
    JOIN Tickets t ON ep.event_id = t.event_id
    JOIN Attendees a ON t.attendee_id = a.attendee_id
    WHERE a.attendee_id IN (
        SELECT t2.attendee_id
        FROM Tickets t2
        JOIN Merchandise m ON t2.event_id = m.event_id
    )
''')
result = [row[0] for row in cursor.fetchall()]
conn.close()
"""
    },
    {
        "input": "Which sponsors funded events that were not cancelled and attended by attendees who also purchased merchandise for other events?",
        "binder_program": """
import sqlite3
conn = sqlite3.connect('{db_name}')
cursor = conn.cursor()
cursor.execute('''
    SELECT DISTINCT s.name
    FROM Sponsors s
    JOIN Event_Sponsors es ON s.sponsor_id = es.sponsor_id
    JOIN Tickets t ON es.event_id = t.event_id
    LEFT JOIN Cancellations c ON t.ticket_id = c.ticket_id
    JOIN Attendees a ON t.attendee_id = a.attendee_id
    WHERE c.cancellation_id IS NULL
    AND a.attendee_id IN (
        SELECT t2.attendee_id
        FROM Tickets t2
        JOIN Merchandise m ON t2.event_id = m.event_id
    )
''')
result = [row[0] for row in cursor.fetchall()]
conn.close()
"""
    },
    {
        "input": "Which datasets were used in experiments that started after January 1, 2020?",
        "binder_program": """
import sqlite3
conn = sqlite3.connect('{db_name}')
cursor = conn.cursor()
cursor.execute('''
    SELECT DISTINCT d.name
    FROM Datasets d
    JOIN Experiments e ON d.dataset_id = e.dataset_id
    WHERE e.start_date > '2020-01-01'
''')
result = [row[0] for row in cursor.fetchall()]
conn.close()
"""
    }
]

# Function to generate database name from schema filename
def get_db_name(schema_file):
    # Convert filename to lowercase, replace spaces and special chars, add .db extension
    base_name = re.sub(r'[^\w\s]', '_', schema_file.replace('.json', '')).lower()
    return f"{base_name}.db"

# Function to set up the SQLite database
def setup_database(schema_data, db_name):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute("PRAGMA foreign_keys = ON")  # Enable foreign keys
    missing_tables = []

    for table in schema_data["tables"]:
        # Create table with TEXT columns and primary key
        columns = ", ".join([f"{col} TEXT" for col in table["columns"]])
        primary_key = table.get("primary_key", "")
        if primary_key:
            columns += f", PRIMARY KEY ({primary_key})"
        create_query = f"CREATE TABLE IF NOT EXISTS {table['name']} ({columns})"
        print(f"Creating table: {table['name']} with query: {create_query}")  # Debug
        try:
            cursor.execute(create_query)
        except Exception as e:
            print(f"Error creating table {table['name']}: {e}")
            missing_tables.append(table['name'])
            continue

        # Insert data
        data = table.get("data", [])
        if data:
            placeholders = ", ".join(["?" for _ in table["columns"]])
            insert_query = f"INSERT INTO {table['name']} ({', '.join(table['columns'])}) VALUES ({placeholders})"
            print(f"Inserting {len(data)} rows into {table['name']}")  # Debug
            try:
                cursor.executemany(insert_query, data)
                print(f"Loaded {len(data)} rows into {table['name']}")
            except Exception as e:
                print(f"Error inserting into {table['name']}: {e}")
                missing_tables.append(table['name'])

    conn.commit()

    # Verify tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = [row[0] for row in cursor.fetchall()]
    print(f"Tables in {db_name}: {tables}")  # Debug
    for table in schema_data["tables"]:
        if table['name'] not in tables:
            missing_tables.append(table['name'])
            print(f"Warning: Table {table['name']} not found in database")
    if missing_tables:
        print(f"Missing tables: {missing_tables}")
    conn.close()
    return missing_tables

# Function to generate BINDER program using Gemini
def generate_binder_program(input_question, examples, schema_data, db_name):
    schema_desc = "Database schema:\n"
    for table in schema_data["tables"]:
        schema_desc += f"Table: {table['name']} ({', '.join(table['columns'])})\n"
        schema_desc += f"Primary Key: {table.get('primary_key', 'None')}\n"
        schema_desc += f"Foreign Keys: {', '.join(table.get('foreign_keys', ['None']))}\n"
        schema_desc += f"Column Descriptions: {json.dumps(table.get('column_descriptions', {}))}\n\n"

    # No reasoning steps included
    prompt = f"""
Generate a Python program using sqlite3 to query a database and answer the question.
Use the database name '{db_name}'.
{schema_desc}
Return only the Python code, no explanations, and assign the query result to a 'result' variable.
Always return results as a list, even for single-row queries (e.g., ['value']).
Do not include code block markers like ```python or ```.

Examples:
"""
    for ex in examples:
        # Replace placeholder with actual db_name
        program = ex['binder_program'].format(db_name=db_name)
        prompt += f"Input: {ex['input']}\nProgram:\n{program}\n\n"
    prompt += f"Input: {input_question}\nProgram:\n"

    resumption_count = 0
    max_resumption = 3
    while resumption_count < max_resumption:
        try:
            response = model.generate_content(
                prompt,
                safety_settings={
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
                }
            )
            print(f"API response type: {type(response)}")
            print(f"Response has 'text' attribute: {hasattr(response, 'text')}")
            if not hasattr(response, 'text'):
                print(f"Error: Response has no 'text' attribute: {response}")
                return None
            print(f"Response text: {response.text[:100]}...")  # First 100 chars
            if not isinstance(response.text, str):
                print(f"Error: response.text is not a string: {type(response.text)}")
                return None
            code = response.text.strip()
            code = re.sub(r'^```(?:python)?\n', '', code, flags=re.MULTILINE)
            code = re.sub(r'\n```$', '', code)
            code = '\n'.join(line for line in code.splitlines() if line.strip())
            code = code.strip()
            return code
        except Exception as e:
            print(f"Error generating BINDER program: {e}")
            resumption_count += 1
            if resumption_count == max_resumption:
                print("Max retries reached. Check API key, client version, or Gemini service.")
                return None

# Function to execute BINDER program
def execute_binder_program(binder_program):
    if not binder_program:
        return {"error": "No valid program generated"}
    print("\nExecuting BINDER Program...\n")
    print("Generated Program:")
    print(binder_program)
    try:
        local_namespace = {}
        exec(binder_program, {}, local_namespace)
        result = local_namespace.get('result', [])
        return {"result": result}
    except Exception as e:
        print(f"Error executing program: {e}")
        return {"error": str(e)}

# Main function
def main():
    schema_dir = "/content/binder_schemas"
    schema_files = sorted([f for f in os.listdir(schema_dir) if f.endswith('.json')])
    if not schema_files:
        print("No schema files found in /content/binder_schemas")
        return

    print("Available schema files:")
    for i, file in enumerate(schema_files, 1):
        print(f"{i}. {file}")
    while True:
        try:
            choice = int(input("Enter the number of the schema file to use (e.g., 1): "))
            if 1 <= choice <= len(schema_files):
                break
            print(f"Please enter a number between 1 and {len(schema_files)}")
        except ValueError:
            print("Please enter a valid number")

    schema_file = schema_files[choice - 1]
    schema_path = os.path.join(schema_dir, schema_file)
    print(f"Selected: {schema_path}")

    with open(schema_path, 'r') as f:
        schema_data = json.load(f)

    # Set database name based on schema filename
    db_name = get_db_name(schema_file)
    print(f"Using database: {db_name}")
    missing_tables = setup_database(schema_data, db_name)
    if missing_tables:
        print(f"Warning: Some tables failed to create: {missing_tables}")

    # Verify database contents
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = [row[0] for row in cursor.fetchall()]
    print("Available tables:", tables)
    for table in schema_data["tables"]:
        try:
            cursor.execute(f"SELECT count(*) FROM {table['name']}")
            count = cursor.fetchone()[0]
            print(f"Rows in {table['name']}: {count}")
        except Exception as e:
            print(f"Error checking rows in {table['name']}: {e}")
    conn.close()

    while True:
        question = input("\nEnter your question (or 'quit' to stop): ")
        if question.lower() == 'quit':
            print("Goodbye! The toy box is closed.")
            break

        print("\nGenerating BINDER Program...\n")
        binder_program = generate_binder_program(question, in_context_examples, schema_data, db_name)
        answer = execute_binder_program(binder_program)

        # Format output as requested
        print(f"Final Answer: {answer}")
        found_expected = False
        for q in schema_data.get("questions", []):
            if q["question"].lower() == question.lower():
                expected = q.get("answer", [])
                actual = answer.get("result", None) if "result" in answer else None
                if actual == expected:
                    print("Validation: Answer matches expected!")
                else:
                    print(f"Validation: Incorrect Answer. Expected: {expected}, Actual: {actual}")
                found_expected = True
                break
        if not found_expected:
            print("Validation: No expected answer found in schema.")

if __name__ == "__main__":
    main()

Found valid model: models/gemini-1.5-flash
Using model: models/gemini-1.5-flash
Available schema files:
1. Travel & Logistics.json
Enter the number of the schema file to use (e.g., 1): 1
Selected: /content/binder_schemas/Travel & Logistics.json
Using database: travel _ logistics.db
Creating table: Flights with query: CREATE TABLE IF NOT EXISTS Flights (flight_id TEXT, flight_number TEXT, departure_airport_id TEXT, arrival_airport_id TEXT, departure_time TEXT, duration_minutes TEXT, PRIMARY KEY (flight_id))
Inserting 5 rows into Flights
Loaded 5 rows into Flights
Creating table: Passengers with query: CREATE TABLE IF NOT EXISTS Passengers (passenger_id TEXT, name TEXT, birth_year TEXT, nationality TEXT, PRIMARY KEY (passenger_id))
Inserting 5 rows into Passengers
Loaded 5 rows into Passengers
Creating table: Bookings with query: CREATE TABLE IF NOT EXISTS Bookings (booking_id TEXT, passenger_id TEXT, flight_id TEXT, booking_date TEXT, seat_number TEXT, PRIMARY KEY (booking_id))
Insertin